In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px

import yfinance as yf


In [2]:
pd.set_option('display.precision',4)

In [3]:
def generate_portfolios(returns, numPortfolios, riskFreeRate=0, riskFreeAsset=False, shortSelling=False):
    tickers = returns.columns.tolist()
    mean_returns = returns.mean() * 252
    cov_matrix = returns.cov() * 252

    # If the risk-free asset is included in the portfolio, add it to mean_returns and cov_matrix
    if riskFreeAsset:
        tickers = tickers + ['RF']
        # Concatenate the risk-free rate to the mean_returns
        mean_returns = np.append(mean_returns, riskFreeRate)
        # Expand the covariance matrix to include the risk-free rate
        cov_matrix = np.vstack([cov_matrix, np.zeros(nAssets-1)])
        cov_matrix = np.hstack([cov_matrix, np.zeros(nAssets).reshape(-1,1)])

    nAssets = len(tickers)
    
    # Create an empty DataFrame to store the results
    portfolios = pd.DataFrame(columns=[ticker+' weight' for ticker in tickers] + ['Return', 'Risk', 'Sharpe Ratio'], index=range(numPortfolios), dtype=float)

    # Generate random weights and calculate the expected return, volatility and Sharpe ratio
    for i in range(numPortfolios):
        weights = np.random.random(nAssets)
        weights /= np.sum(weights)
        portfolios.loc[i, [ticker+' weight' for ticker in tickers]] = weights

        # Calculate the expected return
        portfolios.loc[i, 'Return'] = np.dot(weights, mean_returns)

        # Calculate the expected volatility
        portfolios.loc[i, 'Risk'] = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Calculate the Sharpe ratio
    portfolios['Sharpe Ratio'] = (portfolios['Return'] - riskFreeRate) / portfolios['Risk']

    return portfolios
    
def evaluate_portfolio(mc_portfolios, index, data, initialValue):
    portfolio = mc_portfolios.loc[index]
    tickers = data.columns
    nShares = portfolio[[ticker+' weight' for ticker in tickers]].rename({ticker+' weight' : ticker for ticker in tickers})*initialValue/data.iloc[0]
    portfolio_value = nShares.dot(data.T)
    return portfolio_value

def evaluate_asset(tickers, index, data, initialValue):
    asset = data.iloc[:, index] if len(tickers) > 1 else data
    nShares = initialValue/asset.iloc[0]
    asset_value = nShares*asset
    return asset_value

def minimum_variance_portfolio(mean_returns, cov_matrix):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    w = C_inv.dot(u) / u.T.dot(C_inv).dot(u)
    mu = m.T.dot(w)
    sigma = np.sqrt(w.T.dot(C).dot(w))

    return mu, sigma

def minimum_variance_line(mean_returns, cov_matrix):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    D_mat = np.array([[u.T.dot(C_inv).dot(u), u.T.dot(C_inv).dot(m)],
                      [m.T.dot(C_inv).dot(u), m.T.dot(C_inv).dot(m)]])
    D = np.linalg.det(D_mat)
    a = (u.T.dot(C_inv).dot(u)*C_inv.dot(m) - u.T.dot(C_inv).dot(m)*C_inv.dot(u)) / D
    b = (m.T.dot(C_inv).dot(m)*C_inv.dot(u) - m.T.dot(C_inv).dot(u)*C_inv.dot(m)) / D

    # Calculate the minimum variance line
    mu = np.linspace(0, 0.4, 100)
    w_mu = np.zeros((nAssets, len(mu)))
    sigma_mu = np.zeros(len(mu))
    for i, param in enumerate(mu):
        w_mu[:,i] = a*param + b
        sigma_mu[i] = np.sqrt(w_mu[:,i].T.dot(C).dot(w_mu[:,i]))

    return mu, sigma_mu

def market_portfolio(mean_returns, cov_matrix, riskFreeRate=0):
    nAssets = len(mean_returns)
    m = mean_returns
    C = cov_matrix
    u = np.ones(nAssets)

    # Intermediate calculations
    C_inv = np.linalg.inv(C)
    w = C_inv.dot(m - riskFreeRate*u) / (m - riskFreeRate*u).T.dot(C_inv).dot(u)
    mu = m.T.dot(w)
    sigma = np.sqrt(w.T.dot(C).dot(w))

    return mu, sigma

    

In [4]:
# Get a list of symbols from FTSEMIB index
ftsemib = pd.read_html('https://en.wikipedia.org/wiki/FTSE_MIB')[1]
ftsemib['ICB Sector'] = ftsemib['ICB Sector'].str.extract(r'\((.*?)\)', expand=False).fillna(ftsemib['ICB Sector'])

## Get list of assets

In [16]:
tickers = ['ENI.MI', 'ENEL.MI', 'STMMI.MI', 'LDO.MI', 'A2A.MI',]
# Sort the tickers
tickers.sort()
riskFreeRate = 0.04


In [17]:

start = dt.date(2013, 1, 1)
end = dt.date.today()

data = yf.download(tickers, start=start, end=end)['Adj Close']

[*********************100%***********************]  2 of 2 completed


In [18]:
data.head()

,LDO.MI,STMMI.MI
Date,,
2013-01-02,4.1212,4.1395
2013-01-03,4.2421,4.1987
2013-01-04,4.2457,4.1358
2013-01-07,4.2256,4.2358
2013-01-08,4.1889,4.3172


In [19]:
# Split the data into in-sample and out-of-sample
sample_end = dt.date(2022, 12, 31)
inSample_data = data.loc[:sample_end]
outOfSample_data = data.loc[sample_end:]

In [20]:
# Get the daily returns
returns = inSample_data.pct_change().dropna()

# Collect drifts and standard deviations in the columns of a single DataFrame
tickers_df = pd.DataFrame({'Return': returns.mean() * 252, 'Risk': np.sqrt(np.diag(returns.cov() * 252))}, index=tickers).rename_axis('Ticker')
tickers_df.loc['Risk Free Rate'] = [riskFreeRate, 0]


In [21]:
# Get number of assets from the returns DataFrame
nAssets = len(returns.columns)
nAssets = len(tickers_df)

In [22]:
mean_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252

# # Concatenate the risk-free rate to the mean_returns
# mean_returns = np.append(mean_returns, riskFreeRate)
# cov_matrix = np.append(cov_matrix, [np.zeros(nAssets-1)], axis=0)
# cov_matrix = np.hstack([cov_matrix, np.zeros(nAssets).reshape(-1,1)])

# Minimum variance line
mu, sigma_mu = minimum_variance_line(mean_returns, cov_matrix)
# Minimum variance portfolio
min_var_mu, min_var_sigma = minimum_variance_portfolio(mean_returns, cov_matrix)
# Market portfolio
market_mu, market_sigma = market_portfolio(mean_returns, cov_matrix, riskFreeRate)

Add minimum variance line too

In [23]:
# Plot the mean return and standard deviation with plotly
fig = px.scatter(tickers_df, x='Risk', y='Return', text=tickers_df.index)
fig.update_traces(textposition='top center')
fig.update_layout(title='Mean return vs Standard deviation', xaxis_title='Standard deviation', yaxis_title='Mean return')
fig.add_scatter(x=sigma_mu, y=mu, mode='lines', name='Minimum variance line')
fig.add_scatter(x=[min_var_sigma], y=[min_var_mu], mode='markers', name='Minimum variance portfolio', marker=dict(size=10, color='red'))
fig.add_scatter(x=[market_sigma], y=[market_mu], mode='markers', name='Market portfolio', marker=dict(size=10, color='green'))
fig.add_scatter(x=[0,market_sigma], y=[riskFreeRate,market_mu], mode='lines', name='Capital market line', line=dict(color='green',))
fig.show()


In [24]:
numPortfolios = 10000

mc_portfolios = generate_portfolios(returns, numPortfolios, riskFreeRate=riskFreeRate)

# Plot the portfolios
fig = px.scatter(mc_portfolios, x='Risk', y='Return', color='Sharpe Ratio',  hover_data={**{ticker +' weight': ':.2f' for ticker in tickers}, **{'Return': ':.2f', 'Risk': ':.2f', 'Sharpe Ratio': ':.2f'}}, opacity=0.5,)

fig.update_layout(title='Portfolios', xaxis_title='Standard deviation', yaxis_title='Mean return')
fig.add_scatter(x=tickers_df['Risk'], y=tickers_df['Return'], mode='markers', marker=dict(size=7.5, color='black',),showlegend=False, name='Tickers', text = [f'<b>{index}</b> <br>Standard deviation: {vol:.2f}<br>Expected return: {ret:.2f}' for index, vol, ret in zip(tickers_df.index, tickers_df['Risk'], tickers_df['Return'])],hoverinfo='text')
fig.add_scatter(x=sigma_mu, y=mu, mode='lines', name='Minimum variance line')
fig.show()


In [25]:
# Select sample portfolio
index = 256
sample_portfolio = mc_portfolios.loc[index]
# Determine the performance of the portfolio with out-of-sample data
initialValue = 100
samplePortfolio_value = evaluate_portfolio(mc_portfolios, index, outOfSample_data, initialValue)

fig = px.line(samplePortfolio_value, title='Portfolio value')
fig.update_layout(yaxis_title='Portfolio value')
fig.show()


Compare in-sample and out-of-sample returns